In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [27]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [14]:
pd.read_sql(
    """
    SELECT
        'CRIM' AS COL,
        COUNT(CRIM) AS NULL_COUNT
    FROM boston
    WHERE CRIM IS NULL
    UNION
    SELECT
        'ZN' AS COL,
        COUNT(ZN) AS NULL_COUNT
    FROM boston
    WHERE ZN IS NULL
    UNION
    SELECT
        'INDUS' AS COL,
        COUNT(INDUS) AS NULL_COUNT
    FROM boston
    WHERE INDUS IS NULL
    UNION
    SELECT
        'CHAS' AS COL,
        COUNT(CHAS) AS NULL_COUNT
    FROM boston
    WHERE CHAS IS NULL
    UNION
    SELECT
        'NOX' AS COL,
        COUNT(NOX) AS NULL_COUNT
    FROM boston
    WHERE NOX IS NULL
    """,
    con,
)

,COL,NULL_COUNT
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


In [17]:
# проверим полученные значения средствами pandas
data[['CHAS', 'CRIM', 'INDUS', 'NOX', 'ZN']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CHAS    506 non-null    float64
 1   CRIM    506 non-null    float64
 2   INDUS   506 non-null    float64
 3   NOX     506 non-null    float64
 4   ZN      506 non-null    float64
dtypes: float64(5)
memory usage: 19.9 KB


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [16]:
pd.read_sql(
    """
    SELECT
        'CRIM' AS COL,
        COUNT(DISTINCT CRIM) AS DIST_COUNT
    FROM boston
    UNION
    SELECT
        'ZN' AS COL,
        COUNT(DISTINCT ZN) AS DIST_COUNT
    FROM boston
    UNION
    SELECT
        'INDUS' AS COL,
        COUNT(DISTINCT INDUS) AS DIST_COUNT
    FROM boston
    UNION
    SELECT
        'CHAS' AS COL,
        COUNT(DISTINCT CHAS) AS DIST_COUNT
    FROM boston
    UNION
    SELECT
        'NOX' AS COL,
        COUNT(DISTINCT NOX) AS DIST_COUNT
    FROM boston
    """,
    con,
)

,COL,DIST_COUNT
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


In [22]:
# проверим полученные значения средствами pandas
data[['CHAS', 'CRIM', 'INDUS', 'NOX', 'ZN']].nunique()

CHAS       2
CRIM     504
INDUS     76
NOX       81
ZN        26
dtype: int64

3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [43]:
pd.read_sql(
    """
    WITH STATS AS (
        SELECT
            'CRIM' AS COL,
            MIN(CRIM) AS MIN_VALUE,
            (SELECT AVG(CRIM) FROM (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 2 OFFSET 252)) AS MED_VALUE
        FROM boston
        UNION
        SELECT
            'ZN' AS COL,
            MIN(ZN) AS MIN_VALUE,
            (SELECT AVG(ZN) FROM (SELECT ZN FROM boston ORDER BY ZN LIMIT 2 OFFSET 252)) AS MED_VALUE
        FROM boston
        UNION
        SELECT
            'INDUS' AS COL,
            MIN(INDUS) AS MIN_VALUE,
            (SELECT AVG(INDUS) FROM (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 2 OFFSET 252)) AS MED_VALUE
        FROM boston
        UNION
        SELECT
            'CHAS' AS COL,
            MIN(CHAS) AS MIN_VALUE,
            (SELECT AVG(CHAS) FROM (SELECT CHAS FROM boston ORDER BY CHAS LIMIT 2 OFFSET 252)) AS MED_VALUE
        FROM boston
        UNION
        SELECT
            'NOX' AS COL,
            MIN(NOX) AS MIN_VALUE,
            (SELECT AVG(NOX) FROM (SELECT NOX FROM boston ORDER BY NOX LIMIT 2 OFFSET 252)) AS MED_VALUE
        FROM boston
    )

    SELECT
        COL
    FROM STATS
    WHERE MIN_VALUE = MED_VALUE
    """,
    con,
)

,COL
0,CHAS
1,ZN


In [71]:
# проверим полученные значения средствами pandas
data[['CHAS', 'CRIM', 'INDUS', 'NOX', 'ZN']].median()

CHAS     0.00000
CRIM     0.25651
INDUS    9.69000
NOX      0.53800
ZN       0.00000
dtype: float64

In [44]:
data[['CHAS', 'CRIM', 'INDUS', 'NOX', 'ZN']].min()

CHAS     0.00000
CRIM     0.00632
INDUS    0.46000
NOX      0.38500
ZN       0.00000
dtype: float64

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [41]:
pd.read_sql(
    """
    SELECT
        'CRIM' AS COL,
        MIN(CRIM) AS MIN_VALUE,
        (SELECT AVG(CRIM) FROM (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 2 OFFSET 252)) AS MED_VALUE
    FROM boston
    """,
    con,
)

,COL,MIN_VALUE,MED_VALUE
0,CRIM,0.00632,0.25651


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [82]:
pd.read_sql(
    """
    SELECT
        LSTAT,
        MEDV,
        RANK() OVER (ORDER BY LSTAT)
    FROM boston
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    """,
    con,
)

,LSTAT,MEDV,RANK() OVER (ORDER BY LSTAT)
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [50]:
pd.read_sql(
    """
    SELECT
        'YES' AS RIVER,
        (SELECT AVG(MEDV) FROM boston WHERE CHAS = 1) AS AVG_PRICE
    UNION
    SELECT
        'NO' AS RIVER,
        (SELECT AVG(MEDV) FROM boston WHERE CHAS = 0) AS AVG_PRICE
    """,
    con,
)

,RIVER,AVG_PRICE
0,NO,22.093843
1,YES,28.440000


In [51]:
# проверим полученные значения средствами pandas
data.groupby('CHAS').agg({'MEDV': 'mean'})

,MEDV
CHAS,
0.0,22.093843
1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [56]:
pd.read_sql(
    """
    WITH STATS AS (
        SELECT
            'CRIM' AS COL,
            (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) AS AVG_1,
            (SELECT AVG(CRIM) FROM boston WHERE CHAS = 0) AS AVG_0
        UNION
        SELECT
            'ZN' AS COL,
            (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) AS AVG_1,
            (SELECT AVG(ZN) FROM boston WHERE CHAS = 0) AS AVG_0
        UNION
        SELECT
            'INDUS' AS COL,
            (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) AS AVG_1,
            (SELECT AVG(INDUS) FROM boston WHERE CHAS = 0) AS AVG_0
        UNION
        SELECT
            'CHAS' AS COL,
            (SELECT AVG(CHAS) FROM boston WHERE CHAS = 1) AS AVG_1,
            (SELECT AVG(CHAS) FROM boston WHERE CHAS = 0) AS AVG_0
        UNION
        SELECT
            'NOX' AS COL,
            (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) AS AVG_1,
            (SELECT AVG(NOX) FROM boston WHERE CHAS = 0) AS AVG_0
    )

    SELECT
        COL
    FROM STATS
    WHERE AVG_1 > AVG_0
    """,
    con,
)

,COL
0,CHAS
1,INDUS
2,NOX


In [55]:
# проверим полученные значения средствами pandas
data[['CHAS', 'CRIM', 'INDUS', 'NOX', 'ZN']].groupby('CHAS').mean()

,CRIM,INDUS,NOX,ZN
CHAS,,,,
0.0,3.723215,11.019193,0.551817,11.634820
1.0,1.851670,12.719143,0.593426,7.714286


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [70]:
pd.read_sql(
    """
    SELECT
        10 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 1 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 1 / 10 - 1) AS NOX
    UNION
    SELECT
        20 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 2 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 2 / 10 - 1) AS NOX
    UNION
    SELECT
        30 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 3 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 3 / 10 - 1) AS NOX
    UNION
    SELECT
        40 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 4 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 4 / 10 - 1) AS NOX
    UNION
    SELECT
        50 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 5 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 5 / 10 - 1) AS NOX
    UNION
    SELECT
        60 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 6 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 6 / 10 - 1) AS NOX
    UNION
    SELECT
        70 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 7 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 7 / 10 - 1) AS NOX
    UNION
    SELECT
        80 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 8 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 8 / 10 - 1) AS NOX
    UNION
    SELECT
        90 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 9 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 9 / 10 - 1) AS NOX
    UNION
    SELECT
        100 AS PERC,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 10 / 10 - 1) AS INDUS,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) * 10 / 10 - 1) AS NOX
    """,
    con,
)

,PERC,INDUS,NOX
0,10,2.89,0.426
1,20,4.15,0.442
2,30,5.86,0.469
3,40,7.07,0.507
4,50,9.69,0.538
5,60,12.83,0.575
6,70,18.10,0.605
7,80,18.10,0.668
8,90,19.58,0.713
9,100,27.74,0.871


In [76]:
# проверим полученные значения средствами pandas
data[['INDUS', 'NOX']].quantile([x/10 for x in range(11)], numeric_only=False)

,INDUS,NOX
0.0,0.46,0.385
0.1,2.91,0.427
0.2,4.39,0.442
0.3,5.96,0.472
0.4,7.38,0.507
0.5,9.69,0.538
0.6,12.83,0.575
0.7,18.10,0.605
0.8,18.10,0.668
0.9,19.58,0.713
